# Part2: Cleaning/Formatting Website Data


*Milestone Objective:* This milestone aims to perform data transformations and cleansing on website data related to U.S. States Populations, Land Area, and Population Density (https://www.states101.com/populations). The goal is to create a clean and usable dataset for further analysis by executing at least five specific data transformation steps.

### Transformation Steps and Code Outline

In [1]:
# Step #0: Scrape and Parse Data from Website into DataFrame

# In this step, extracted the population data from the website using the requests and BeautifulSoup libraries.
# fetched the HTML, identified the table containing population data, and converted it into a structured pandas DataFrame.

import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the page to scrape
url = 'https://www.states101.com/populations'

# Send a GET request to the webpage
response = requests.get(url)

# Check if the request was successful (status code 200 means OK)
if response.status_code == 200:
    # Parse the HTML content of the webpage using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the table with the population data by ID ('populationTable')
    table = soup.find('table', id='populationTable')
    
    # Find all the rows in the table, skipping the header row (index 1 onward)
    rows = table.find_all('tr')[1:]   

     # Initialize an empty list to hold the scraped data
    data = []

    # Iterate through the rows of the table
    for row in rows:
        # Extract all 'td' (table data) elements from the row
        cols = row.find_all('td')

         # Check if the number of columns is as expected (at least 4 columns)
        if len(cols) < 4:
            print(f"Found {len(cols)} columns in row: {row}")
            continue # Skip the row if columns are fewer than expected

        # Extract and clean the state name
        state = cols[0].text.strip() # Strip any extra spaces or newlines

        # Extract and clean the population value
        # Remove commas, extra quotes, and newlines, then remove the rank (e.g., '(1st)')
        population = cols[1].text.strip().split(' ')[0].replace(',', '').replace('"', '').replace('\n', '').strip()
        population = population.split('(')[0].strip()  # Remove anything inside parentheses (rank)
        population = int(population)  # Convert the cleaned population value to an integer

        # Extract and clean the land area value
        # Similar cleaning steps for land area as for population
        land_area = cols[2].text.strip().split(' ')[0].replace(',', '').replace('"', '').replace('\n', '').strip()
        land_area = land_area.split('(')[0].strip()  # Remove rank
        land_area = float(land_area)  # Convert to float

        # Extract and clean the density value
        # Again, apply similar cleaning to remove unnecessary characters and rank
        density = cols[3].text.strip().split(' ')[0].replace(',', '').replace('"', '').replace('\n', '').strip()
        density = density.split('(')[0].strip()  # Remove rank
        density = float(density)   # Convert to float for numerical calculations

        # Append the cleaned data (state, population, land area, density) to the list
        data.append([state, population, land_area, density])

    # Create a DataFrame from the cleaned list of data
    df = pd.DataFrame(data, columns=['State', 'Population', 'Land Area (sq mi)', 'Density (people per sq mi)'])

    # Display the first few rows of the DataFrame to ensure data is loaded correctly
    print(df.head())

else:
     # If the request was unsuccessful, print the status code (e.g., 404, 500)
    print(f"Failed to retrieve data: {response.status_code}")


        State  Population  Land Area (sq mi)  Density (people per sq mi)
0  California    38803000          155779.03                       249.1
1       Texas    26957000          261231.59                       103.2
2     Florida    19893000           53624.55                       371.0
3    New York    19746000           47126.45                       419.0
4    Illinois    12881000           55518.76                       232.0


In [2]:
# Step #1: Standardize column names for consistency

# This step converts all column names to lowercase and replaces spaces with underscores.
# This makes it easier to reference the columns later in the analysis and ensures consistency.
df.columns = df.columns.str.lower().str.replace(' ', '_')

# Display the updated column names to verify the change
print(df.columns)

Index(['state', 'population', 'land_area_(sq_mi)',
       'density_(people_per_sq_mi)'],
      dtype='object')


In [3]:
# Step #2: Fix casing for state names

# Normalizing the casing of state names ensures uniformity, which is essential for tasks like mapping state names to their codes and avoiding errors in comparisons later.
df['state'] = df['state'].str.title()

# Display the DataFrame to verify the change
print(df.head())

        state  population  land_area_(sq_mi)  density_(people_per_sq_mi)
0  California    38803000          155779.03                       249.1
1       Texas    26957000          261231.59                       103.2
2     Florida    19893000           53624.55                       371.0
3    New York    19746000           47126.45                       419.0
4    Illinois    12881000           55518.76                       232.0


In [4]:
# Step #3: Identify and remove duplicate entries

# Removing duplicates early in the process prevents them from affecting our analysis. 
# This step ensures that each record in our dataset is unique before we perform more complex transformations.

# Count the number of rows before removing duplicates
initial_count = df.shape[0]

# Remove duplicates based on all columns
df = df.drop_duplicates()

# Count the number of rows after removing duplicates
final_count = df.shape[0]

# Display the number of duplicates removed
print(f"Number of duplicate rows removed: {initial_count - final_count}")

# Display the DataFrame to verify the change
print(df.head())

Number of duplicate rows removed: 0
        state  population  land_area_(sq_mi)  density_(people_per_sq_mi)
0  California    38803000          155779.03                       249.1
1       Texas    26957000          261231.59                       103.2
2     Florida    19893000           53624.55                       371.0
3    New York    19746000           47126.45                       419.0
4    Illinois    12881000           55518.76                       232.0


In [5]:
# Step #4: Create a State Code Mapping Using the Dictionary

# With clean state names in a consistent format, we can now reliably map them to their corresponding state codes.
# Did this to ensure consistency across all 3 data sources (i.e., the flat file, website data, and API),
# as the state code will be the key used for joining in the later analysis.

# Mapping of full state names to their two-character codes
state_mapping = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY"
}

# Apply the state mapping to create a new column with state codes
df['state_code'] = df['state'].map(state_mapping)

# Display the updated DataFrame to ensure that state codes have been added correctly
print(df.head())

        state  population  land_area_(sq_mi)  density_(people_per_sq_mi)  \
0  California    38803000          155779.03                       249.1   
1       Texas    26957000          261231.59                       103.2   
2     Florida    19893000           53624.55                       371.0   
3    New York    19746000           47126.45                       419.0   
4    Illinois    12881000           55518.76                       232.0   

  state_code  
0         CA  
1         TX  
2         FL  
3         NY  
4         IL  


In [6]:
# Step #5: Filter out any non-U.S. states

# After ensuring the data is unique and consistently formatted, we can now filter out any non-U.S. states.
# This is important as non-U.S. states or territories could be present in the data and would skew analysis if not removed.

# Display the count before filtering
count_before_filter = df.shape[0]
print(f"Count before filtering: {count_before_filter} rows")

# Filter out any non-U.S. states by checking for non-null values in the 'state_code' column.
df = df[df['state_code'].notna()]

# Display the count after filtering to see how many rows remain
count_after_filter = df.shape[0]
print(f"Count after filtering: {count_after_filter} rows")

# Final Output: Print the cleaned DataFrame to verify the result
print(df.head())

Count before filtering: 56 rows
Count after filtering: 50 rows
        state  population  land_area_(sq_mi)  density_(people_per_sq_mi)  \
0  California    38803000          155779.03                       249.1   
1       Texas    26957000          261231.59                       103.2   
2     Florida    19893000           53624.55                       371.0   
3    New York    19746000           47126.45                       419.0   
4    Illinois    12881000           55518.76                       232.0   

  state_code  
0         CA  
1         TX  
2         FL  
3         NY  
4         IL  


### Ethical Implications of Data Wrangling from the Website Data 
*https://www.states101.com/populations*

In the process of cleaning and transforming the population data from the website, several steps were applied, such as standardizing column names, normalizing the casing of state names, removing duplicate entries, mapping state names to two-letter codes, and filtering out non-U.S. states. These changes enhanced the dataset for analysis, but they also come with ethical considerations. The data was sourced from the U.S. Census Bureau via Census.gov, which is a credible and authoritative source. However, legal and regulatory guidelines regarding web scraping must still be considered, as terms of service agreements may restrict such activities. The transformations, such as excluding non-U.S. states, could introduce risks by potentially omitting important data. Additionally, assumptions made during data cleaning, such as the consistency of state names, could lead to inaccuracies. Since the data was sourced from a credible government site, its accuracy is more likely, but it’s important to respect any restrictions on data use. To mitigate these ethical risks, careful consideration of the terms of use, clear documentation of assumptions, and ongoing review of the data’s integrity are necessary to ensure compliance and reliability.